In [1]:
import pandas as pd
import requests as r

In [78]:
base_url = 'https://dom.mingkh.ru/'

In [8]:
dd = pd.read_html('https://dom.mingkh.ru/moskva/moskva/houses',extract_links='body')[0]

In [72]:
dd[['Адрес','home_url']] = dd['Адрес'].apply('|'.join).str.split('|',expand=True)

In [75]:
dd.loc[0,'home_url']

'/moskva/moskva/916691'

In [89]:
def parse_home_page(home_url):
    df_list = pd.read_html(home_url)
    home_data = pd.concat(df_list,axis=0)\
                    .iloc[:,:3].dropna(thresh=1)\
                        .set_index(0).fillna('')\
                            .astype(str).apply(''.join,axis=1)\
                                .rename(home_url.split('/')[-1])
    return home_data

In [96]:
h1 = parse_home_page(base_url+dd.loc[0,'home_url'])
h2 = parse_home_page(base_url+dd.loc[1,'home_url'])

In [97]:
h1.to_frame().join(h2)

,916691,422302
0,,
Вентиляция,Вытяжная вентиляция,Вытяжная вентиляция
Внутренние стены,Внутренние стены,Внутренние стены
Водоотведение,Центральное,Центральное
Газоснабжение,Центральное,Центральное
Год ввода в эксплуатацию,1968,1959
Горячее водоснабжение,Закрытая с приготовлением горячей воды на ЦТП,Закрытая с приготовлением горячей воды на ЦТП
Детская площадка,Есть,Есть
Дом признан аварийным,Нет,Нет
"Износ здания, %",24,48
